In [1]:
from zipfile import ZipFile
import pandas as pd
import numpy as np

# Загрузка данных

In [2]:
def gen_parcer(f):
    for line in f.readlines():
        yield line.decode().strip().split("\t")

In [3]:
with ZipFile('data.zip') as datazip:
    with datazip.open('train.tsv') as f:
        pd_train = pd.DataFrame(gen_parcer(f),
                                columns=["context_id", "context_2", "context_1", "context_0", "reply_id", "reply",
                                         "label", "confidence"])
    with datazip.open('public.tsv') as f:
         pd_public = pd.DataFrame(gen_parcer(f),
                                  columns=["context_id", "context_2", "context_1", "context_0", "reply_id", "reply"])

In [4]:
pd_train.head()

,context_id,context_2,context_1,context_0,reply_id,reply,label,confidence
0,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",0,не могу .,good,0.8753516175
1,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",1,"нет , звонить буду я .",neutral,0.9009682113
2,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",2,"слушай , я не мог уйти .",bad,0.8843202145
3,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",3,я не прекращу звонить .,good,0.9825304673
4,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",4,я звоню им .,good,0.8380535096


In [5]:
pd_public.head()

,context_id,context_2,context_1,context_0,reply_id,reply
0,138920940977,"знаешь , я иногда подумываю , что тебе надо пр...",не - а .,нет ?,0,неа .
1,138920940977,"знаешь , я иногда подумываю , что тебе надо пр...",не - а .,нет ?,1,"нет , не хочу ."
2,138920940977,"знаешь , я иногда подумываю , что тебе надо пр...",не - а .,нет ?,2,нет .
3,138920940977,"знаешь , я иногда подумываю , что тебе надо пр...",не - а .,нет ?,3,"конечно , нет ."
4,138920940977,"знаешь , я иногда подумываю , что тебе надо пр...",не - а .,нет ?,4,"разумеется , нет ."


In [6]:
pd_train.count()

context_id    97533
context_2     97533
context_1     97533
context_0     97533
reply_id      97533
reply         97533
label         97533
confidence    97533
dtype: int64

In [7]:
pd_public.count()

context_id    9968
context_2     9968
context_1     9968
context_0     9968
reply_id      9968
reply         9968
dtype: int64

In [8]:
target_encoder = {'bad': 0, 'good': 2, 'neutral': 1}  
pd_train.label = pd_train.label.map(target_encoder)

In [9]:
pd_data = pd.concat([pd_train, pd_public], axis=0).reset_index(drop=True)
pd_data.label.fillna(-1, inplace=True)
pd_data.confidence.fillna(0.0, inplace=True)

# Что за данные

In [10]:
pd_train.shape, pd_public.shape, pd_data.shape

((97533, 8), (9968, 6), (107501, 8))

In [11]:
pd_data.groupby("label")["context_id"].count() / len(pd_data)

label
-1.0    0.092725
 0.0    0.323439
 1.0    0.104687
 2.0    0.479149
Name: context_id, dtype: float64

# Простой бейзлайн

In [12]:
import pickle 

In [13]:
import re

w2v_fpath = "w2v/tenth.norm-sz500-w7-cb0-it5-min5.w2v"
w2v = gensim.models.KeyedVectors.load_word2vec_format(w2v_fpath, binary=True, unicode_errors='ignore')
w2v.init_sims(replace=True)

In [14]:
w2v = pickle.load(open("w2v/w2vs.pickle", "rb"))

In [15]:
s_words = pd_data[["context_2", "context_1", "context_0", "reply"]] \
    .apply(lambda x: " ".join(x), axis=1)

In [16]:
def clean_text(x):
    x = x.lower()
    x = x.replace("ё", "е")
    return x

In [18]:
empty_vector = np.zeros(500)
def condext_to_vec(x):
    words = re.findall(r"\b(\w+)\b", clean_text(x))
    vectors = sum([w2v[word] for word in words if word in w2v])
    return (empty_vector + vectors).reshape(1, -1)

In [19]:
def not_in_vec(x):
    words = re.findall(r"\b([a-zа-я]+)\b", clean_text(x))
    words = [word for word in words if word not in w2v]
    return " ".join(words)

In [20]:
s_strange = s_words.apply(not_in_vec)

In [21]:
len(s_strange[~s_strange.eq("")])

1084

In [22]:
s_words.loc[2600]

'ќе шучу . " ты разрешишь мне вернутьс€ в тюрьму ? я начинаю уставать , это так унизительно . когда я возбуждаюсь , я издаю такого рода звуки . раньше ты не лаял .'

In [23]:
s_strange[~s_strange.eq("")]

193         вудхалл
624          фабрей
867           ворус
868           ворус
869           ворус
870           ворус
871           ворус
872           ворус
1109         волмов
1110         волмов
1111         волмов
1112         волмов
1113         волмов
1114         волмов
1952       эрствайл
2166          дэнно
2346         гендзи
2549       оливерио
3025        огустин
3026        огустин
3027        огустин
3028        огустин
3029        огустин
3030        огустин
3232         изобэл
3301         тоетие
3302         тоетие
3303         тоетие
3304         тоетие
3305         тоетие
            ...    
104418     костанза
104419     костанза
104420     кастиэль
104455        тофер
104730        дэнно
105017      ротстин
105711         шадо
105874       пиджей
105875       пиджей
105876       пиджей
105877       пиджей
105878       пиджей
105879       пиджей
106052    сулбаимеи
106053        cэнди
106075      адалинд
106277      погугли
106284          тьi
106849     скиттеры


## features

In [24]:
def to_features_w2v(df):
    s_words = df[["context_2", "context_1", "context_0"]].apply(lambda x: " ".join(x), axis=1)
    context = np.concatenate(s_words.apply(condext_to_vec).tolist())
    
    s_words = df["reply"]
    reply = np.concatenate(s_words.apply(condext_to_vec).tolist())
    
    features = np.concatenate([context, reply], axis=1)
    return features

## crossval

In [25]:
from tqdm import tqdm_notebook

In [26]:
from sklearn.model_selection import StratifiedKFold

In [27]:
skf = StratifiedKFold(n_splits=3)

In [28]:
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.ensemble import RandomForestRegressor

In [29]:
from sklearn.metrics import mean_absolute_error

In [30]:
def calc_ndcg_score(df, y_predict):
    df = df.copy()
    df["predict"] = y_predict
    #dcg
    df.sort_values(["context_id", "predict"], ascending=[True, False], inplace=True)
    df["row_count"] = df.groupby("context_id").cumcount()+1
    df["dcg"] = df["label"] / np.log2(df["row_count"] + 1)
    #idcg
    df.sort_values(["context_id", "label"], ascending=[True, False], inplace=True)
    df["row_count"] = df.groupby("context_id").cumcount()+1
    df["idcg"] = df["label"] / np.log2(df["row_count"] + 1)
    
    df_context = df.groupby("context_id").sum()
    return (df_context["dcg"] / df_context["idcg"]).mean() * 100000

In [114]:
def do_cross_val(model_reg, to_features, to_target, model_fit_params=None):
    mae_errors = []
    ndcg_scores = []

    for _train, _test in tqdm_notebook(skf.split(pd_train.index, pd_train.label), total=3):
        i_train = pd_train.iloc[_train]
        i_test = pd_train.iloc[_test]

        X_train = to_features(i_train)
        X_test = to_features(i_test)
        Y_train = to_target(i_train)
        Y_test = to_target(i_test)

        model_reg.load_weights("model_nn.w")
        model_reg.fit(X_train, Y_train, **model_fit_params)
        Y_predict = model_reg.predict(X_test)
        mae_errors.append(mean_absolute_error(Y_test, Y_predict))
        ndcg_scores.append(calc_ndcg_score(i_test, Y_predict))

    print(mae_errors)
    print(ndcg_scores)

## target

In [32]:
trg_label = lambda x: x.label.values

In [33]:
trg_label_confidence = lambda x: (x.label.values - 1) * x.confidence.values.astype(np.float)

## Нормальные варианты

In [34]:
do_cross_val(LinearRegression(), to_features_w2v, trg_label)

A Jupyter Widget


[0.82638449406819026, 0.82574022422909588, 0.82666949630383568]
[86306.57817832676, 86285.70686972419, 86404.02531996512]


In [35]:
do_cross_val(LinearRegression(), to_features_w2v, trg_label_confidence)

A Jupyter Widget


[0.71481938513885968, 0.71375232297887958, 0.71354418000758502]
[86336.8461095529, 86280.83807429724, 86467.65361220425]


# Сеточка

In [82]:
from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM, Dropout

In [116]:
model = Sequential([
    Dense(1000, input_dim=1000),
    Activation('relu'),
    Dropout(0.5),
    Dense(50),
    Activation('relu'),
    Dense(1),
    Activation('linear'),
])

model.compile(optimizer='adam',
              loss='mean_squared_error')
model.save_weights("model_nn.w")

In [118]:
do_cross_val(model, to_features_w2v, trg_label_confidence,
            {"epochs": 10, "batch_size": 256, "verbose": 2, "validation_split": 0.1})

A Jupyter Widget

Train on 58518 samples, validate on 6503 samples
Epoch 1/10
 - 18s - loss: 0.6488 - val_loss: 0.6246
Epoch 2/10
 - 17s - loss: 0.6253 - val_loss: 0.6176
Epoch 3/10
 - 19s - loss: 0.6180 - val_loss: 0.6142
Epoch 4/10
 - 19s - loss: 0.6115 - val_loss: 0.6090
Epoch 5/10
 - 18s - loss: 0.6064 - val_loss: 0.6122
Epoch 6/10
 - 17s - loss: 0.6010 - val_loss: 0.6035
Epoch 7/10
 - 18s - loss: 0.5970 - val_loss: 0.6074
Epoch 8/10
 - 18s - loss: 0.5922 - val_loss: 0.6035
Epoch 9/10
 - 18s - loss: 0.5882 - val_loss: 0.6039
Epoch 10/10
 - 17s - loss: 0.5837 - val_loss: 0.6089
Train on 58519 samples, validate on 6503 samples
Epoch 1/10
 - 17s - loss: 0.7011 - val_loss: 0.6476
Epoch 2/10
 - 19s - loss: 0.6463 - val_loss: 0.6389
Epoch 3/10
 - 18s - loss: 0.6416 - val_loss: 0.6333
Epoch 4/10
 - 18s - loss: 0.6374 - val_loss: 0.6411
Epoch 5/10
 - 18s - loss: 0.6329 - val_loss: 0.6260
Epoch 6/10
 - 18s - loss: 0.6294 - val_loss: 0.6262
Epoch 7/10
 - 17s - loss: 0.6256 - val_loss: 0.6217
Epoch 8/10
 - 17s

# Отладка

In [37]:
_train, _test = next(skf.split(pd_train.index, pd_train.label))
i_train = pd_train.iloc[_train]
i_test = pd_train.iloc[_test]

In [39]:
i_train

,context_id,context_2,context_1,context_0,reply_id,reply,label,confidence
32457,93647138840824,все не так уж плохо .,сделайте себя полезными .,и это вытянем .,1,будет трудно .,2,0.909092886
32458,93647138840824,все не так уж плохо .,сделайте себя полезными .,и это вытянем .,2,"высоко поднять мы не сможем , так что не медли .",2,0.8059265579
32459,93683303278579,хорошо .,может будет подмога .,капитан лэнс ?,0,"да , мисс гибсон .",2,0.9899184337
32460,93683303278579,хорошо .,может будет подмога .,капитан лэнс ?,1,отставной капитан флота .,2,0.8011402464
32461,93683303278579,хорошо .,может будет подмога .,капитан лэнс ?,2,капитан риггз .,2,0.8468712695
32462,93683303278579,хорошо .,может будет подмога .,капитан лэнс ?,3,капитан башир .,2,0.9221820127
32463,93683303278579,хорошо .,может будет подмога .,капитан лэнс ?,4,"нет , капитан дженкинс .",2,0.9927068992
32464,93683303278579,хорошо .,может будет подмога .,капитан лэнс ?,5,капитан даффи .,2,0.9389570234
32465,93695962253917,вперед . привет .,"у нас квест и первое задание , повесить его по...","а кто он , я его не знаю .",0,"я не знаю , как его зовут .",2,0.9389397554
32466,93695962253917,вперед . привет .,"у нас квест и первое задание , повесить его по...","а кто он , я его не знаю .",1,"царь иудеи , некто христос .",2,0.7832186689


In [103]:
X_train = to_features_w2v(i_train)
X_test = to_features_w2v(i_test)
Y_train = trg_label_confidence(i_train)
Y_test = trg_label_confidence(i_test)

In [109]:
model.fit(X_train, Y_train, epochs=10, batch_size=256, verbose=2, validation_split=0.2)

Train on 52016 samples, validate on 13005 samples
Epoch 1/10
 - 16s - loss: 0.5819 - val_loss: 0.6090
Epoch 2/10
 - 16s - loss: 0.5776 - val_loss: 0.6067
Epoch 3/10
 - 16s - loss: 0.5743 - val_loss: 0.6061
Epoch 4/10
 - 16s - loss: 0.5680 - val_loss: 0.6102
Epoch 5/10
 - 16s - loss: 0.5661 - val_loss: 0.6084
Epoch 6/10
 - 16s - loss: 0.5601 - val_loss: 0.6073
Epoch 7/10
 - 16s - loss: 0.5571 - val_loss: 0.6085
Epoch 8/10
 - 17s - loss: 0.5527 - val_loss: 0.6110
Epoch 9/10
 - 17s - loss: 0.5476 - val_loss: 0.6043
Epoch 10/10
 - 16s - loss: 0.5434 - val_loss: 0.6089


In [110]:
Y_predict = model.predict(X_test)

In [111]:
print(mean_absolute_error(Y_test, Y_predict))
print(calc_ndcg_score(i_test, Y_predict))

0.69282322577
86568.10480536966


In [113]:
model.load_weights("model_nn.w")

TypeError: 'list' object is not callable

# Submit

In [119]:
def do_submit(submission_name, model_reg, to_features, to_target, model_fit_params=None):
    i_train = pd_train
    i_test = pd_public.copy()

    X_train = to_features(i_train)
    X_test = to_features(i_test)
    Y_train = to_target(i_train)

    model_reg.fit(X_train, Y_train, **model_fit_params)
    Y_predict = model_reg.predict(X_test)

    i_test["predict"] = Y_predict
    i_test.sort_values(["context_id", "predict"], ascending=[True, False], inplace=True)

    i_test[["context_id", "reply_id"]].to_csv(submission_name, index=False, header=False, sep="\t")
    
    return i_test

In [120]:
do_submit("w2v_nn_mlp.tsv", model, to_features_w2v, trg_label_confidence,
          {"epochs": 10, "batch_size": 256, "verbose": 2, "validation_split": 0.1}).head(15)

Train on 87779 samples, validate on 9754 samples
Epoch 1/10
 - 28s - loss: 0.6091 - val_loss: 0.6085
Epoch 2/10
 - 26s - loss: 0.6053 - val_loss: 0.6071
Epoch 3/10
 - 26s - loss: 0.6017 - val_loss: 0.6035
Epoch 4/10
 - 26s - loss: 0.5986 - val_loss: 0.6038
Epoch 5/10
 - 26s - loss: 0.5942 - val_loss: 0.6013
Epoch 6/10
 - 27s - loss: 0.5901 - val_loss: 0.6014
Epoch 7/10
 - 27s - loss: 0.5877 - val_loss: 0.5985
Epoch 8/10
 - 27s - loss: 0.5846 - val_loss: 0.5995
Epoch 9/10
 - 26s - loss: 0.5820 - val_loss: 0.5971
Epoch 10/10
 - 28s - loss: 0.5782 - val_loss: 0.6016


,context_id,context_2,context_1,context_0,reply_id,reply,predict
3479,100097508986637,,,выключай их,4,"пытаюсь , сэр .",0.640513
3478,100097508986637,,,выключай их,3,я пытаюсь !,0.464230
3476,100097508986637,,,выключай их,1,"ты не выключишь , тогда € сам выключу",0.265819
3475,100097508986637,,,выключай их,0,выключить что ?,0.057735
3480,100097508986637,,,выключай их,5,выключите,-0.237809
3477,100097508986637,,,выключай их,2,выключить ?,-0.328770
3486,100149747456986,,как вы здесь оказались ?,не надо меня тянуть !,5,"что "" не надо "" ? не надо меня .",0.478471
3484,100149747456986,,как вы здесь оказались ?,не надо меня тянуть !,3,мне уже легче .,0.422974
3483,100149747456986,,как вы здесь оказались ?,не надо меня тянуть !,2,я не могу позволить вам уйти .,0.385519
3481,100149747456986,,как вы здесь оказались ?,не надо меня тянуть !,0,ладно .,0.176011
